In [1]:
import numpy as np
import pandas as pd
import os
import requests
import json
import urllib
import tensorflow as tf
import tensorflow_datasets
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Lambda, Dense, dot
import tensorflow.keras as keras
%config Completer.use_jedi =False

In [23]:
'''Make sure you have a steam account before running this code
Data includes draft data from patches-('7.24'","'7.25'","'7.26'","'7.27'","'7.28'","'7.29'","'7.30')

Note:Patches-'7.24','7.25','7.26' have only 12 bans whereas the rest of them have 14.Appended two -1's  to make the
array consistent
'''
wd=os.getcwd()
pnb = os.path.join(wd,"picks_and_bans")
dj = os.path.join(wd,"draft_json")
if not os.path.exists(pnb):
    os.mkdir(pnb)
if not os.path.exists(dj):
    os.mkdir(dj)

heroes_df=pd.read_csv('hero_df.csv')
patches=["'7.24'","'7.25'","'7.26'","'7.27'","'7.28'","'7.29'","'7.30'"]
for i in patches:
    query_string="Select matches.picks_bans from matches join match_patch using(match_id) where match_patch.patch like "+i
    quoted_string=urllib.parse.quote(query_string)
    json_response = requests.get(
        'https://api.opendota.com/api/explorer?sql=' + quoted_string).json()
    f=open('draft_json\\'+i.lstrip('\'').rstrip('\'')+".json","w")
    json.dump(json_response,f)
    f.close()

In [10]:
picks_list = []
bans_list = []
hero_df=pd.read_csv('hero_df.csv')
json_files='draft_json/'
for name in ("7.24.json","7.25.json","7.26.json","7.27.json","7.28.json","7.29.json","7.30.json"):
    with open(json_files+name) as f:
        json_response=json.load(f)
    rowsdf=pd.DataFrame(json_response['rows'])
    if name in ["7.27.json","7.28.json","7.29.json","7.30.json"]:
        instances=24
    else:
        instances=22
    for i in rowsdf['picks_bans']:
        picks=[]
        bans=[]
        try:
            for j in range(instances):
                current=hero_df.loc[hero_df['id']==i[j]['hero_id']]['sid'].values[0]
                
                if i[j]['is_pick']:
                    picks.append(current)
                else:
                    bans.append(current)
                
                    
        except:
            continue
        if name in ["7.27.json","7.28.json","7.29.json","7.30.json"]:
            if len(picks)<10 or len(bans)<14:
                continue
            else:
                picks_list.append(picks)
                bans_list.append(bans)
        else:
            if len(picks)<10 or len(bans)<12:
                continue
            else:
                
                bans.append(-1)
                bans.append(-1)
                picks_list.append(picks)
                bans_list.append(bans)
        
        
picks_array=np.array(picks_list).astype('int32')
bans_array=np.array(bans_list).astype('int32')
np.save('picks_and_bans\\picks_array',picks_array)
np.save('picks_and_bans\\bans_array',bans_array)
            
            

In [4]:
picks_array=np.load('picks_array.npy')
bans_array=np.load('bans_array.npy')